In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen
from tqdm import tqdm_notebook
import datetime

In [2]:
def get_metacritic():

    df_meta = pd.DataFrame(columns=['artist', 'album', 'meta', 'user', 'date'])

    for page in tqdm_notebook(range(50+1)):
        url = 'http://www.metacritic.com/browse/albums/release-date/available/date?page='+str(page)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        base = soup.find_all("li", {"class": "product"})

        for i in range(len(base)):

            row = base[i]

            artist = row.select(".stat.product_artist span")[1].get_text()
            tmp = row.select(".basic_stat.product_title > a")[0].get_text()
            album = tmp[29:-53]
            date = row.select("div.basic_stat.condensed_stats > ul > li.stat.release_date.full_release_date > span.data")[0].get_text()
            meta = float(row.select(".metascore_w")[0].get_text())
            user = row.select(".product_avguserscore span")[1].get_text()

            input_data = {
                'artist' : artist,
                'album' : album,
                'date' : date,
                'meta' : meta,
                'user' : user,
            }

            df_meta.loc[len(df_meta)] = input_data
        
    return df_meta

In [3]:
metacritic_score = get_metacritic()

HBox(children=(IntProgress(value=0, max=51), HTML(value='')))

In [4]:
print(metacritic_score.shape)
metacritic_score.head()

(6800, 5)


,artist,album,meta,user,date
0,Sleep,The Sciences,86.0,tbd,"Apr 20, 2018"
1,Lord Huron,Vide Noir,70.0,8.1,"Apr 20, 2018"
2,Alexis Taylor,Beautiful Thing,71.0,tbd,"Apr 20, 2018"
3,Old Crow Medicine Show,Volunteer,85.0,tbd,"Apr 20, 2018"
4,Drinks,Hippo Lite,74.0,tbd,"Apr 20, 2018"


In [5]:
metacritic_score.to_csv("../data/scrapy-metacritic-1118", index=False)